In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from python_utils import *
from tqdm.notebook import tqdm, trange

### Case 1: $F = \mathbb Q(E[12])$

In [2]:
e0 = 12
def get_ramification_dataset1(l):
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':divisors(12*l)}
    ep_range[2] = {'good':[e for e in divisors(pow(2,8) * pow(3,2)) if e%2 == 0], 'multi':[e for e in divisors(24*l) if e%2 == 0]}
    ep_range[3] = {'good':[e for e in divisors(pow(2,6) * pow(3,2)) if e%2 == 0], 'multi':[e for e in divisors(48*l) if e%2 == 0]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[(l-1)*e for e in divisors(12*l)]}
    return ep_range

# compute log-volume
my_log_volumes1 = dict()
for p in tqdm(primes(150)):
    if p >= 5 and p <= 30:
        my_log_volumes1[p] = compute_log_volume(get_ramification_dataset1, e0, p)
        print(p, my_log_volumes1[p])
    elif p > 30:
        my_log_volumes1[p] = compute_log_volume_fast(get_ramification_dataset1, e0, p)

  0%|          | 0/35 [00:00<?, ?it/s]

5 133.92056029892038
7 116.89711856627233
11 126.66197956597263
13 135.53407835194733
17 157.00550521728516
19 167.61314166656254
23 190.32667244864382
29 225.4548740511201


In [3]:
e0 = 12
def get_impossible_interval1(log_volumes,S,k,t0,is_t_ge_t0 = False):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []

    # define p_*, u_*, n_*
    p0 = S[0]
    p0_apos = 2
    u0 = t0
    n0 = 1728
    if is_t_ge_t0:
        n1_S = t0
    else: # if t = t0
        n1_S = p0 * t0

    nk_S = prod(S[:k])
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] 
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = log(2)
    #  define b_1, b'_1 b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S, a1/u0)
    b2 = a1/u0 * log(n0) + a2 + a1*a3+ (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p0_apos)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound1(log_volumes,t0,is_t_ge_t0 = False):
    if is_t_ge_t0:
        my_primes = [p for p in primes(150) if p >= 11 and p != 13]
    else:
        my_primes = [p for p in primes(150) if p >= 11 and p != 13 and t0%p != 0]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N)
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval1(log_volumes,S,k,t0,is_t_ge_t0)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    return res

# upper bound for t >= t0 = 1000
res = get_upper_bound1(my_log_volumes1,1000, is_t_ge_t0=True)
print(1000,f"{(res[-1][0] + log(1728)) + 1e-3: .3f}",res)
print(res[-1][0] + log(1728), 1000 * log(13), res[-1][0] + log(1728) > 1000 * log(13)) # => 2560 > log(z^t) > 1000 * log(13) -- a contradiction!

# upper bound for different 11 <= t = t0 < 1000
bound_dict = dict()
for t in trange(11,1000):
    bound = get_upper_bound1(my_log_volumes1,t)[-1][0] + log(1728)
    bound_dict[t] = bound
    # note that if t1 | t, we have z^t = (z^{t/t1})^t1 is a t1-th power, hence an upper bound for t1 is also an upper bound for t
    real_bound = min([bound_dict[t1] for t1 in divisors(t) if t1 >= 11])
    if t in (11,13,17,19) or real_bound > 2750:
        print(t,f"{real_bound + 1e-3: .3f}")

# since t is not divided by 6,7,8,9,10,15, we have 
# log(z^t)<3730 for t=11; log(z^t)<3085 for t=13; and log(z^t)<2789 for t>=17

1000  2557.398 [[2336.3393294720436, 2462.7519325294857], [2549.9420114608083, 19459245008.35703]]
2557.3967314101724 2564.9493574615367 False


  0%|          | 0/989 [00:00<?, ?it/s]

11  3729.700
12  3426.173
13  3084.533
14  2971.457
15  2828.736
16  2750.098
17  2706.577
19  2641.589
23  2788.928
46  2753.185


### Case 2: $F=Q(E[2q])$

In [4]:
e0 = 2
def get_ramification_dataset2(l,q):
    # l \nmid q*(q*q-1)
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':divisors(2*l)}
    ep_range[2] = {'good':[e for e in divisors(pow(2,5) * pow(3,2)) if e%2 == 0], 'multi':[e for e in divisors(8*q*l) if e%2 == 0]}
    ep_range[3] = {'good':[1], 'multi':[2*e for e in divisors(q*l)]}
    ep_range[q] = {'good':[q-1,q*(q-1),q*q-1], 'multi':[(q-1)*e for e in divisors(2*l)]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[(l-1)*e for e in divisors(2*l)]}
    return ep_range

# compute log-volume
my_log_volumes2 = dict()
for q in tqdm(primes(1000)):
    def get_ramification_dataset2_for_fixed_q(l):
        return get_ramification_dataset2(l,q)
    # compute log-volume for fixed q
    my_log_volumes2[q] = dict()
    for p in primes(60):
        if p < 5:
            continue
        if p < 30 and q < 100:
            my_log_volumes2[q][p] = compute_log_volume(get_ramification_dataset2_for_fixed_q, e0, p) + log(2)
        else:
            my_log_volumes2[q][p] = compute_log_volume_fast(get_ramification_dataset2_for_fixed_q, e0, p) + log(2)

  0%|          | 0/168 [00:00<?, ?it/s]

In [5]:
e0 = 2
def get_impossible_interval2(log_volumes,S,k,t0,is_t_ge_t0 = False):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []

    # define p_*, u_*, n_*
    p0 = S[0]
    p0_apos = 2
    u0 = t0
    n0 = 27
    n1_S = p0 * t0

    nk_S = prod(S[:k])
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[q][l]
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = log(2)
    #  define b_1, b'_1 b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S, a1/u0)
    b2 = a1/u0 * log(n0) + a2 + a1*a3+ (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p0_apos)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound2(log_volumes,t, q):
    my_primes = [p for p in primes(60) if p >= 11 and p != 13 and t%p != 0 and q*(q*q-1) % p != 0]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N)
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval2(log_volumes,S,k,t)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    return res

# upper bound for different 11 <= t = t0 < 100
bound_dict = dict()
real_bound_dict = dict()
for t in trange(11,1000):
    if t%6 == 0 or t%8 == 0 or t%9 == 0:
        continue
    bound = -1
    for q in divisors(t):
        if q < 5 or not is_prime(q):
            continue
        bound = max(bound, get_upper_bound2(my_log_volumes2,t,q)[-1][0] + log(27))
    bound_dict[t] = bound
    real_bound_dict[t] = min([bound_dict[t1] for t1 in divisors(t) if t1 >= 11])

  0%|          | 0/989 [00:00<?, ?it/s]

In [6]:
for t in real_bound_dict.keys():
    real_bound = real_bound_dict[t]
    if t > 330 and real_bound + 1e-3 > t * log(13):
        # the possible t > 340
        print(t,f"{real_bound + 1e-3: .3f}")
        # hence for t > 340, we have t < 353 or t = 373

331  940.456
332  881.253
334  888.256
335  1006.402
337  903.588
338  884.212
339  913.835
341  957.461
345  924.061
346  916.390
347  925.798
349  925.901
353  941.599
361  939.171
370  969.155
373  1018.416


In [7]:
for t in real_bound_dict.keys():
    real_bound = real_bound_dict[t]
    if t > 353 and t != 373:
        continue
    if t in (11,13,17,19) or real_bound + 1e-3 > 981:
        print(t,f"{real_bound + 1e-3: .3f}")
# hence we have log(z^t)<952 for t != 11,13,229,307,373

11  1275.949
13  1082.939
17  994.082
19  970.914
23  1003.064
37  983.174
43  983.715
46  995.449
67  1010.407
69  992.934
92  991.680
115  990.930
134  1007.900
161  990.073
201  1007.067
230  989.432
253  989.296
268  1006.651
299  989.087
335  1006.402
373  1018.416


### Lemma ...

In [103]:
def a1(p):
    return 6 * (p*p + 5*p) / (p*p + p - 12) * (1 - 1 / 2 / p)

def a2(p,q):
    return 6 * my_log_volumes2[q][p] + a1(p) * log(6 * p)

def possible_divisors_of_z(t):
    # find all the p >= 5 such that p may divided z
    possible_l_and_p_bound = []
    for q in [q for q in divisors(t) if q >= 5 and is_prime(q)]:
        S = [l for l in primes(100) if l>=11 and l!=13 and t%l!=0 and (q*q-1)%l!=0]
        for l in (S[0], S[1], S[2]):
            p_bound = exp( a2(l,q) / (t - a1(l)))
            possible_l_and_p_bound.append((l,p_bound))
    # 
    possible_p = []
    for p in primes(1000):
        flag = True
        for (l,p_bound) in possible_l_and_p_bound:
            if p != l and p > p_bound:
                flag = False
                break
        if flag:
            possible_p.append(p)
    return possible_p

for t in range(80,400):
    if len([t0 for t0 in (6,7,8,9,10,15) if t%t0==0]) > 0:
        continue
    res = possible_divisors_of_z(t)
    print(t, res)

82 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]
83 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
85 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
86 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
87 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41]
89 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
92 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79]
93 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
94 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
95 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
97 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
101 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
103 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
106 [2, 3, 5, 7, 11, 13, 17, 19]
107 [2, 3, 5, 7, 11, 13, 17, 19, 23]
109 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
111 [2, 3, 5, 7, 11, 13, 17]
113 [2, 3, 5, 7, 11, 13, 17, 19]
115 [2, 3, 5, 7, 11, 13]
116 [2, 3, 5, 7, 11, 13]
118 [2, 3, 5, 7, 11, 13]
121 [2, 3, 5, 7, 11, 1

In [115]:

def upper_bound_of_coprime_to_6_part_of_z(t):
    max_p = max(possible_divisors_of_z(t))
    z6_bound = 999
    for q in [q for q in divisors(t) if q >= 5 and is_prime(q)]:
        S = [l for l in primes(100) if l>=11 and l!=13 and t%l!=0 and (q*q-1)%l!=0]
        l0 = min([l for l in S if l > max_p])
        tmp_z6_bound = exp( a2(l0,q) / (t - a1(l0)))
        # we have z6 <= tmp_z6_bound0
        l1, l2 = S[0],S[1]
        # if l1 * l2 > tmp_z6_bound0, then z is not divided by l1 or l2, hence one of them can be used to estimate z6
        if l1 * l2 > tmp_z6_bound:
            tmp_z6_bound1 = exp( a2(l1,q) / (t - a1(l1)))
            tmp_z6_bound2 = exp( a2(l2,q) / (t - a1(l2)))
            tmp_z6_bound = min(tmp_z6_bound, max(tmp_z6_bound1, tmp_z6_bound2))
        z6_bound = min(z6_bound, tmp_z6_bound)
    return z6_bound

for t in range(95,400):
    if len([t0 for t0 in (6,7,8,9,10,15) if t%t0==0]) > 0:
        continue
    res = upper_bound_of_coprime_to_6_part_of_z(t)
    if t>=90 and res > 30:
        print(t, res)


95 35.76227488905235
97 42.5863059348027
101 42.97264792263167
103 39.80856152059753
107 31.596821197495128
109 32.227185875207354
116 34.43504026098007
